<a href="https://colab.research.google.com/github/kovendhan5/deepfake/blob/main/deepfakedetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import cv2
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam

# Step 1: Unzip the dataset
def unzip_dataset(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Example usage:
zip_path = "dataset.zip"
extract_to = "dataset"  # Directory to extract the dataset
unzip_dataset(zip_path, extract_to)

# Step 2: Load the dataset (real and fake frames)
def load_dataset(real_dir, fake_dir):
    data = []
    labels = []
    # Load real frames
    for img_name in os.listdir(real_dir):
        img_path = os.path.join(real_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            processed_img = preprocess_frame(img)
            data.append(processed_img)
            labels.append(0)  # 0 for real
    # Load fake frames
    for img_name in os.listdir(fake_dir):
        img_path = os.path.join(fake_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            processed_img = preprocess_frame(img)
            data.append(processed_img)
            labels.append(1)  # 1 for deepfake
    return np.array(data), np.array(labels)

# Step 3: Preprocess frames
def preprocess_frame(frame):
    frame = cv2.resize(frame, (224, 224))  # Resize to match ResNet50 input size
    frame = frame.astype("float") / 255.0  # Normalize pixel values to [0, 1]
    return img_to_array(frame)

# Step 4: Build the model
def build_deepfake_model():
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    head_model = base_model.output
    head_model = Flatten()(head_model)
    head_model = Dense(512, activation="relu")(head_model)
    head_model = Dropout(0.5)(head_model)  # Dropout for regularization
    head_model = Dense(1, activation="sigmoid")(head_model)  # Binary classification (real or fake)
    model = Model(inputs=base_model.input, outputs=head_model)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Step 5: Train the model
def train_deepfake_model(real_dir, fake_dir, batch_size=32, epochs=20):
    data, labels = load_dataset(real_dir, fake_dir)
    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, random_state=42)
    train_aug = ImageDataGenerator(rotation_range=15, fill_mode="nearest")
    model = build_deepfake_model()
    model.fit(train_aug.flow(trainX, trainY, batch_size=batch_size),
              validation_data=(testX, testY),
              steps_per_epoch=len(trainX) // batch_size,
              epochs=epochs)
    model.save("deepfake_detection_model.h5")
    return model

# Step 6: Load and use the trained model for predictions
def detect_deepfake(model, frame):
    processed_frame = preprocess_frame(frame)
    processed_frame = np.expand_dims(processed_frame, axis=0)  # Add batch dimension
    prediction = model.predict(processed_frame)[0][0]
    return prediction > 0.5  # Return True if prediction indicates deepfake

# Example usage of training
real_dir = "dataset/real"  # Directory with real frames
fake_dir = "dataset/fake"  # Directory with fake frames
model = train_deepfake_model(real_dir, fake_dir)

# Step 7: Implement email alert system
def send_email_alert(victim_name, recipient_email, frame_path):
    sender_email = "deepfakevideosdetection@gmail.com"  # Replace with your email
    sender_password = "Deepfakevideosdetection@5"  # Replace with your email password
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = f"Deepfake Alert: {victim_name}"

    body = f"""
    ALERT: A deepfake was detected for {victim_name} at {datetime.now()}!

    Please take immediate action by choosing one of the following options:

    1. [Report to Social Media Customer Care](https://your_custom_link/report)
    2. [File a Complaint with Cybercrime](https://your_custom_link/file_complaint)

    Best regards,
    Your Deepfake Detection Team
    """
    msg.attach(MIMEText(body, 'html'))

    attachment = open(frame_path, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f"attachment; filename= {os.path.basename(frame_path)}")
    msg.attach(part)

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)
        server.quit()
        print(f"Email alert sent to {recipient_email}!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Step 8: Live video detection with email alerts
def process_video_and_alert(victim_name, recipient_email):
    cap = cv2.VideoCapture(0)  # Webcam capture
    model = load_model("deepfake_detection_model.h5")

    while True:
        ret, frame = cap.read()
        if ret:
            is_deepfake = detect_deepfake(model, frame)
            if is_deepfake:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                frame_path = f"deepfake_{timestamp}.jpg"
                cv2.imwrite(frame_path, frame)
                send_email_alert(victim_name, recipient_email, frame_path)

            cv2.imshow('Live Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage for live detection  and alert
victim_name = "kovendhan"
recipient_email = "kovendhan2535@gmail.com"
process_video_and_alert(victim_name, recipient_email)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# 1. Model Training
def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train):
    input_shape = X_train.shape[1:]
    model = create_model(input_shape)
    model.fit(X_train, y_train, epochs=10, validation_split=0.2)
    return model

# 2. Video Processing
def process_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)
    cap.release()
    return np.array(frames) / 255.0

# 3. Deep Fake Detection
def detect_deepfake(video_path, model):
    processed_frames = process_video(video_path, model)
    predictions = model.predict(processed_frames)
    return np.mean(predictions) > 0.5

# 4. Email Alert System
def send_alert(recipient_email):
    sender_email = "deepfakevideosdetection@gmail.com"
    password = "Deepfakevideosdetection@5"

    message = MIMEMultipart("alternative")
    message["Subject"] = "Deep Fake Alert"
    message["From"] = sender_email
    message["To"] = recipient_email

    text = """
    A deep fake video using your face has been detected.

    Please choose an action:
    1. Report to Social Media Customer Care
    2. Report to Cybercrime Authorities
    """

    html = """
    <html>
    <body>
        <p>A deep fake video using your face has been detected.</p>
        <p>Please choose an action:</p>
        <a href="http://example.com/report_social_media">
            <button>Report to Social Media Customer Care</button>
        </a>
        <a href="http://example.com/report_cybercrime">
            <button>Report to Cybercrime Authorities</button>
        </a>
    </body>
    </html>
    """

    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    message.attach(part1)
    message.attach(part2)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, recipient_email, message.as_string())

# Main execution
if __name__ == "__main__":
    # Assume X_train and y_train are your training data and labels
    # X_train = ...
    # y_train = ...

    # Train the model
    # model = train_model(X_train, y_train)

    # For demonstration, we'll skip training and use a dummy model
    model = create_model((224, 224, 3))

    # Detect deep fake in a video
    video_path = "/content/video.mp4"
    is_deepfake = detect_deepfake(video_path, model)

    if is_deepfake:
        send_alert("kovendhan2535@gmail.com")
        print("Alert sent successfully!")
    else:
        print("No deep fake detected.")

25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-20710ee70afsm14611575ad.162 - gsmtp')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

# 1. Model Training
def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train):
    input_shape = X_train.shape[1:]
    model = create_model(input_shape)
    model.fit(X_train, y_train, epochs=10, validation_split=0.2)
    return model

# 2. Video Processing
def process_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)
    cap.release()
    return np.array(frames) / 255.0

# 3. Deep Fake Detection
def detect_deepfake(video_path, model):
    processed_frames = process_video(video_path, model)
    predictions = model.predict(processed_frames)
    return np.mean(predictions) > 0.5

# 4. Email Alert System
def send_alert(recipient_email):
    sender_email = os.environ.get('SENDER_EMAIL')
    app_password = os.environ.get('APP_PASSWORD')

    if not sender_email or not app_password:
        print("Error: Email credentials not set in environment variables.")
        return

    message = MIMEMultipart("alternative")
    message["Subject"] = "Deep Fake Alert"
    message["From"] = sender_email
    message["To"] = recipient_email

    text = """
    A deep fake video using your face has been detected.

    Please choose an action:
    1. Report to Social Media Customer Care
    2. Report to Cybercrime Authorities
    """

    html = """
    <html>
    <body>
        <p>A deep fake video using your face has been detected.</p>
        <p>Please choose an action:</p>
        <a href="http://example.com/report_social_media">
            <button>Report to Social Media Customer Care</button>
        </a>
        <a href="http://example.com/report_cybercrime">
            <button>Report to Cybercrime Authorities</button>
        </a>
    </body>
    </html>
    """

    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    message.attach(part1)
    message.attach(part2)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("Alert sent successfully!")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

# Main execution
if __name__ == "__main__":
    # Set environment variables for email credentials
    os.environ['SENDER_EMAIL'] = 'deepfakevideosdetection@gmail.com'
    os.environ['APP_PASSWORD'] = 'xjtk lizr pdnj tnvv'

    # Assume X_train and y_train are your training data and labels
    # X_train = ...
    # y_train = ...

    # Train the model
    # model = train_model(X_train, y_train)

    # For demonstration, we'll skip training and use a dummy model
    model = create_model((224, 224, 3))

    # Detect deep fake in a video
    video_path = "/content/video.mp4"
    is_deepfake = detect_deepfake(video_path, model)

    if is_deepfake:
        send_alert("kovendhan2535@gmail.com")
    else:
        print("No deep fake detected.")

25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step
No deep fake detected.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

#creating a Model
def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train):
    input_shape = X_train.shape[1:]
    model = create_model(input_shape)
    model.fit(X_train, y_train, epochs=10, validation_split=0.2)
    return model

#Video Processing
def process_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)
    cap.release()
    return np.array(frames) / 255.0

#Deep Fake Detection
def detect_deepfake(video_path, model):
    processed_frames = process_video(video_path, model)
    predictions = model.predict(processed_frames)
    return np.mean(predictions) > 0.5

#Email Alert System
def send_alert(recipient_email):
    sender_email = os.environ.get('SENDER_EMAIL')
    app_password = os.environ.get('APP_PASSWORD')

    if not sender_email or not app_password:
        print("Error: Email credentials not set in environment variables.")
        return

    message = MIMEMultipart("alternative")
    message["Subject"] = "Deep Fake Alert"
    message["From"] = sender_email
    message["To"] = recipient_email

    text = """
    A deep fake video using your face has been detected.

    Please choose an action:
    1. Report to Social Media Customer Care
    2. Report to Cybercrime Authorities
    """

    html = """
    <html>
    <body>
        <p>A deep fake video using your face has been detected.</p>
        <p>Please choose an action:</p>
        <a href="http://example.com/report_social_media">
            <button>Report to Social Media Customer Care</button>
        </a>
        <a href="http://example.com/report_cybercrime">
            <button>Report to Cybercrime Authorities</button>
        </a>
    </body>
    </html>
    """

    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    message.attach(part1)
    message.attach(part2)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("Alert sent successfully!")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

# Main execution
if __name__ == "__main__":
    #environment variables for email credentials
    os.environ['SENDER_EMAIL'] = 'deepfakevideosdetection@gmail.com'
    os.environ['APP_PASSWORD'] = 'xjtk lizr pdnj tnvv'
    # Train the model
    # model = train_model(X_train, y_train)
    # dummy model
    model = create_model((224, 224, 3))

    # Detect deep fake in a video
    video_path = "/content/video.mp4"
    is_deepfake = detect_deepfake(video_path, model)

    if is_deepfake:
        send_alert("kovendhan2535@gmail.com")
    else:
        print("No deep fake detected.")

In [ ]:
import os
import zipfile
import cv2
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam

# Step 1: Unzip the dataset
def unzip_dataset(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Example usage:
zip_path = "dataset.zip"
extract_to = "dataset"  # Directory to extract the dataset
unzip_dataset(zip_path, extract_to)

# Step 2: Load the dataset (real and fake frames)
def load_dataset(real_dir, fake_dir):
    data = []
    labels = []
    # Load real frames
    for img_name in os.listdir(real_dir):
        img_path = os.path.join(real_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            processed_img = preprocess_frame(img)
            data.append(processed_img)
            labels.append(0)  # 0 for real
    # Load fake frames
    for img_name in os.listdir(fake_dir):
        img_path = os.path.join(fake_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            processed_img = preprocess_frame(img)
            data.append(processed_img)
            labels.append(1)  # 1 for deepfake
    return np.array(data), np.array(labels)

# Step 3: Preprocess frames
def preprocess_frame(frame):
    frame = cv2.resize(frame, (224, 224))  # Resize to match ResNet50 input size
    frame = frame.astype("float") / 255.0  # Normalize pixel values to [0, 1]
    return img_to_array(frame)

# Step 4: Build the model
def build_deepfake_model():
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    head_model = base_model.output
    head_model = Flatten()(head_model)
    head_model = Dense(512, activation="relu")(head_model)
    head_model = Dropout(0.5)(head_model)  # Dropout for regularization
    head_model = Dense(1, activation="sigmoid")(head_model)  # Binary classification (real or fake)
    model = Model(inputs=base_model.input, outputs=head_model)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Step 5: Train the model
def train_deepfake_model(real_dir, fake_dir, batch_size=32, epochs=20):
    data, labels = load_dataset(real_dir, fake_dir)
    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, random_state=42)
    train_aug = ImageDataGenerator(rotation_range=15, fill_mode="nearest")
    model = build_deepfake_model()
    model.fit(train_aug.flow(trainX, trainY, batch_size=batch_size),
              validation_data=(testX, testY),
              steps_per_epoch=len(trainX) // batch_size,
              epochs=epochs)
    model.save("deepfake_detection_model.h5")
    return model

# Step 6: Load and use the trained model for predictions
def detect_deepfake(model, frame):
    processed_frame = preprocess_frame(frame)
    processed_frame = np.expand_dims(processed_frame, axis=0)  # Add batch dimension
    prediction = model.predict(processed_frame)[0][0]
    return prediction > 0.5  # Return True if prediction indicates deepfake

# Example usage of training
real_dir = "dataset/train/real"  # Directory with real frames
fake_dir = "dataset/train/fake"  # Directory with fake frames
model = train_deepfake_model(real_dir, fake_dir)

#Email Alert System
def send_alert(recipient_email):
    sender_email = os.environ.get('SENDER_EMAIL')
    app_password = os.environ.get('APP_PASSWORD')

    if not sender_email or not app_password:
        print("Error: Email credentials not set in environment variables.")
        return

    message = MIMEMultipart("alternative")
    message["Subject"] = "Deep Fake Alert"
    message["From"] = sender_email
    message["To"] = recipient_email

    text = """
    A deep fake video using your face has been detected.

    Please choose an action:
    1. Report to Social Media Customer Care
    2. Report to Cybercrime Authorities
    """

    html = """
    <html>
    <body>
        <p>A deep fake video using your face has been detected.</p>
        <p>Please choose an action:</p>
        <a href="http://example.com/report_social_media">
            <button>Report to Social Media Customer Care</button>
        </a>
        <a href="http://example.com/report_cybercrime">
            <button>Report to Cybercrime Authorities</button>
        </a>
    </body>
    </html>
    """

    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    message.attach(part1)
    message.attach(part2)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("Alert sent successfully!")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

# Main execution
if __name__ == "__main__":
    #environment variables for email credentials
    os.environ['SENDER_EMAIL'] = 'deepfakevideosdetection@gmail.com'
    os.environ['APP_PASSWORD'] = 'xjtk lizr pdnj tnvv'
    # Train the model
    # model = train_model(X_train, y_train)
    # dummy model
    model = create_model((224, 224, 3))

    # Detect deep fake in a video
    video_path = "/content/video.mp4"
    is_deepfake = detect_deepfake(video_path, model)

    if is_deepfake:
        send_alert("kovendhan2535@gmail.com")
    else:
        print("No deep fake detected.")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications
import cv2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

# 1. Enhanced Model Training
def create_model(input_shape):
    base_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train, epochs=20):
    input_shape = X_train.shape[1:]
    model = create_model(input_shape)
    model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, batch_size=32)
    return model

# 2. Improved Video Processing
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = applications.efficientnet.preprocess_input(frame)
    return frame

def detect_faces(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return faces

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        faces = detect_faces(frame)
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = preprocess_frame(face)
            frames.append(face)
    cap.release()
    return np.array(frames)

# 3. Enhanced Deep Fake Detection
def detect_deepfake(video_path, model):
    processed_frames = process_video(video_path)
    if len(processed_frames) == 0:
        print("No faces detected in the video.")
        return False
    predictions = model.predict(processed_frames)
    deepfake_score = np.mean(predictions)
    print(f"Deepfake probability: {deepfake_score:.2f}")
    return deepfake_score > 0.5

# 4. Email Alert System (unchanged)
def send_alert(recipient_email):
    sender_email = os.environ.get('SENDER_EMAIL')
    app_password = os.environ.get('APP_PASSWORD')

    if not sender_email or not app_password:
        print("Error: Email credentials not set in environment variables.")
        return

    message = MIMEMultipart("alternative")
    message["Subject"] = "Deep Fake Alert"
    message["From"] = sender_email
    message["To"] = recipient_email

    text = """
    A deep fake video using your face has been detected.

    Please choose an action:
    1. Report to Social Media Customer Care
    2. Report to Cybercrime Authorities
    """

    html = """
    <html>
    <body>
        <p>A deep fake video using your face has been detected.</p>
        <p>Please choose an action:</p>
        <a href="http://example.com/report_social_media">
            <button>Report to Social Media Customer Care</button>
        </a>
        <a href="http://example.com/report_cybercrime">
            <button>Report to Cybercrime Authorities</button>
        </a>
    </body>
    </html>
    """

    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    message.attach(part1)
    message.attach(part2)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("Alert sent successfully!")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

    # ... (same as before)

# Main execution
if __name__ == "__main__":
    # Set environment variables for email credentials
    os.environ['SENDER_EMAIL'] = 'deepfakevideosdetection@gmail.com'
    os.environ['APP_PASSWORD'] = 'xjtk lizr pdnj tnvv'

    # In a real scenario, you would load your training data here
    # X_train, y_train = load_training_data()
    # model = train_model(X_train, y_train)

    # For demonstration, we'll use a pre-trained model
    model = create_model((224, 224, 3))

    # You should load pre-trained weights here
    # model.load_weights('path_to_your_pretrained_weights.h5')

    video_path = "/content/video.mp4"
    is_deepfake = detect_deepfake(video_path, model)

    if is_deepfake:
        send_alert("kovendhan2535@gmail.com")
    else:
        print("No deep fake detected.")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


KeyboardInterrupt: 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications
import cv2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

# 1. Enhanced Model Training
def create_model(input_shape):
    base_model = applications.EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train, epochs=20, batch_size=32):
    input_shape = X_train.shape[1:]
    model = create_model(input_shape)
    model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, batch_size=batch_size)
    return model

# 2. Improved Video Processing
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (300, 300))
    frame = applications.efficientnet.preprocess_input(frame)
    return frame

def detect_faces(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return faces

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        faces = detect_faces(frame)
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = preprocess_frame(face)
            frames.append(face)
    cap.release()
    return np.array(frames)

# 3. Enhanced Deep Fake Detection
def detect_deepfake(video_path, model):
    processed_frames = process_video(video_path)
    if len(processed_frames) == 0:
        print("No faces detected in the video.")
        return False
    predictions = model.predict(processed_frames)
    deepfake_score = np.mean(predictions)
    print(f"Deepfake probability: {deepfake_score:.2f}")
    return deepfake_score > 0.5

# 4. Email Alert System (unchanged)
def send_alert(recipient_email):
    sender_email = os.environ.get('SENDER_EMAIL')
    app_password = os.environ.get('APP_PASSWORD')

    if not sender_email or not app_password:
        print("Error: Email credentials not set in environment variables.")
        return

    message = MIMEMultipart("alternative")
    message["Subject"] = "Deep Fake Alert"
    message["From"] = sender_email
    message["To"] = recipient_email

    text = """
    A deep fake video using your face has been detected.

    Please choose an action:
    1. Report to Social Media Customer Care
    2. Report to Cybercrime Authorities
    """

    html = """
    <html>
    <body>
        <p>A deep fake video using your face has been detected.</p>
        <p>Please choose an action:</p>
        <a href="http://example.com/report_social_media">
            <button>Report to Social Media Customer Care</button>
        </a>
        <a href="http://example.com/report_cybercrime">
            <button>Report to Cybercrime Authorities</button>
        </a>
    </body>
    </html>
    """

    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    message.attach(part1)
    message.attach(part2)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("Alert sent successfully!")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

    # ... (same as before)


# Main execution
if __name__ == "__main__":
    # Set environment variables for email credentials
    os.environ['SENDER_EMAIL'] = 'deepfakevideosdetection@gmail.com'
    os.environ['APP_PASSWORD'] = 'xjtk lizr pdnj tnvv'

    # Load your training data
    # Replace this with your actual data loading function
    X_train = np.random.rand(1000, 300, 300, 3)
    y_train = np.random.randint(2, size=(1000, 1))

    model = train_model(X_train, y_train, epochs=20, batch_size=32)

    # Save the trained model
    model.save('deepfake_detection_model.h5')

    # Load the saved model
    model = tf.keras.models.load_model('deepfake_detection_model.h5')

    video_path = "/content/video.mp4"
    is_deepfake = detect_deepfake(video_path, model)

    if is_deepfake:
        send_alert("kovendhan2535@gmail.com")
    else:
        print("No deep fake detected.")

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 354s 13s/step - accuracy: 0.4629 - loss: 0.7745 - val_accuracy: 0.4450 - val_loss: 0.6936
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 387s 14s/step - accuracy: 0.5075 - loss: 0.7019 - val_accuracy: 0.4450 - val_loss: 0.7040
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 390s 14s/step - accuracy: 0.4974 - loss: 0.7047 - val_accuracy: 0.4450 - val_loss: 0.6948
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 340s 14s/step - accuracy: 0.5046 - loss: 0.6960 - val_accuracy: 0.4450 - val_loss: 0.6999
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 346s 14s/step - accuracy: 0.4769 - loss: 0.6993 - val_accuracy: 0.4450 - val_loss: 0.6942
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 345s 14s/step - accuracy: 0.5085 - loss: 0.6942 - val_accuracy: 0.4450 - val_loss: 0.6953
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 370s 13s/step - accuracy: 0.4957 - loss: 0.6935 - val_accuracy: 0.4450 - val_loss: 0.6948
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 380s 13s/step -

24/24 ━━━━━━━━━━━━━━━━━━━━ 230s 9s/step
Deepfake probability: 0.56
Alert sent successfully!


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications
import cv2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

# 1. Enhanced Model Training
def create_model(input_shape):
    base_model = applications.EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(X_train, y_train, epochs=20, batch_size=32):
    input_shape = X_train.shape[1:]
    model = create_model(input_shape)
    model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, batch_size=batch_size)
    return model

# 2. Improved Video Processing
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (300, 300))
    frame = applications.efficientnet.preprocess_input(frame)
    return frame

def detect_faces(frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return faces

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        faces = detect_faces(frame)
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = preprocess_frame(face)
            frames.append(face)
    cap.release()
    return np.array(frames)

# 3. Enhanced Deep Fake Detection
def detect_deepfake(video_path, model):
    processed_frames = process_video(video_path)
    if len(processed_frames) == 0:
        print("No faces detected in the video.")
        return False
    predictions = model.predict(processed_frames)
    deepfake_score = np.mean(predictions)
    print(f"Deepfake probability: {deepfake_score:.2f}")
    return deepfake_score > 0.5

# 4. Email Alert System (unchanged)
def send_alert(recipient_email):
    sender_email = os.environ.get('SENDER_EMAIL')
    app_password = os.environ.get('APP_PASSWORD')

    if not sender_email or not app_password:
        print("Error: Email credentials not set in environment variables.")
        return

    message = MIMEMultipart("alternative")
    message["Subject"] = "Deep Fake Alert"
    message["From"] = sender_email
    message["To"] = recipient_email

    text = """
    A deep fake video using your face has been detected.

    Please choose an action:
    1. Report to Social Media Customer Care
    2. Report to Cybercrime Authorities
    """

    html = """
    <html>
    <body>
        <p>A deep fake video using your face has been detected.</p>
        <p>Please choose an action:</p>
        <a href="http://example.com/report_social_media">
            <button>Report to Social Media Customer Care</button>
        </a>
        <a href="http://example.com/report_cybercrime">
            <button>Report to Cybercrime Authorities</button>
        </a>
    </body>
    </html>
    """

    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")

    message.attach(part1)
    message.attach(part2)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, app_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("Alert sent successfully!")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

    # ... (same as before)


# Main execution
if __name__ == "__main__":
    # Set environment variables for email credentials
    os.environ['SENDER_EMAIL'] = 'deepfakevideosdetection@gmail.com'
    os.environ['APP_PASSWORD'] = 'xjtk lizr pdnj tnvv'

    # Load your training data
    # Replace this with your actual data loading function
    #X_train = np.random.rand(1000, 300, 300, 3)
    #y_train = np.random.randint(2, size=(1000, 1))

    #model = train_model(X_train, y_train, epochs=20, batch_size=32)

    # Save the trained model
   # model.save('deepfake_detection_model.h5')

    # Load the saved model
    model = tf.keras.models.load_model('deepfake_detection_model.h5')

    video_path = "/content/video.mp4"
    is_deepfake = detect_deepfake(video_path, model)

    if is_deepfake:
        send_alert("kovendhan2535@gmail.com")
    else:
        print("No deep fake detected.")